In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import (Output, FloatSlider, IntSlider,
                        Box, HBox, VBox, Layout, Checkbox,
                        Button, HTML, Text, Textarea)
from iam_utils import (WidgetParbox, WidgetPlot,
                      WidgetDataDumper, WidgetCodeCheck)
import ase
from ase.io import read, write
import itertools

In [ ]:
import copy # deepcopy
import IPython.display # DisplayHandle

In [ ]:
data_dump = WidgetDataDumper(prefix="ex_05")
display(data_dump)

# Reciprocal lattice

The reciprocal lattice is the Fourier transformation of the lattice in position space. Given a Bravais lattice $ \mathbf{T}_\mathbf{u}$ we can express its reciprocal lattice as

$$
\mathbf{G}_\mathbf{n} = n_1\mathbf{b}_1 + n_2\mathbf{b}_2 + n_3\mathbf{b}_3 \textrm{ under the condition } \mathbf{G}_\mathbf{n}\cdot \mathbf{T}_\mathbf{u} = 2\pi N \textrm{ for a }N\in\mathbb{Z}
$$

For simplicity, we will only consider 2D lattices in the rest of the notebook. The reciprocal lattice vectors can be constructed from the Bravais latitce $\mathbf{T}_\mathbf{u}$ with

$$ \mathbf{b}_1 = 2\pi\frac{\mathbf{R}\mathbf{a}_2}{\mathbf{a}_1\cdot\mathbf{R}\mathbf{a}_2},$$
$$ \mathbf{b}_2 = 2\pi\frac{\mathbf{R}\mathbf{a}_1}{\mathbf{a}_2\cdot\mathbf{R}\mathbf{a}_1},$$
$$ \textrm{where }\mathbf{R}\textrm{ is a }\frac{\pi}2\textrm{ rotation, so }\mathbf{R} = \begin{bmatrix}0 & -1\\ 1 & 0\end{bmatrix}$$

<span style="color:blue"> **08** What are the reciprocal primitive vectors for a 2D cubic Bravais lattice with primitve vectors $(1,0)$ and $(0,1)$?
</span>

In [ ]:
ex08_txt = Textarea("The reciprocal primitive vectors: b_1 = ... , b_2 = ...", layout=Layout(width="100%"))
data_dump.register_field("ex08-answer", ex08_txt, "value")
display(ex08_txt)

# Diffraction - Laue equations

The reciprocal lattice has a few properties which makes it useful to look at it in the context of x-ray diffraction. As you might remember, for a constructive interference of the incident x-ray $\mathbf{k}_\textrm{in}$ within the crystal system and therefore to observe a peak in the intensity of the diffracted x-ray $\mathbf{k}_\textrm{out}$ on a crystal struture the Laue equations must be satisfied

IMAGE DIFFRACTION OF CRYSTAL

$$(\mathbf{k}_\textrm{in}-\mathbf{k}_\textrm{out})\mathbf{a}_1 = 2\pi N_1,$$
$$(\mathbf{k}_\textrm{in}-\mathbf{k}_\textrm{out})\mathbf{a}_2 = 2\pi N_2,$$
$$(\mathbf{k}_\textrm{in}-\mathbf{k}_\textrm{out})\mathbf{a}_3 = 2\pi N_3,$$

for some integers $N_1,N_2,N_3\in\mathbb{Z}$.

The difference in the incoming and outgoing wave vectors is called *scattering vector* $\mathbf{q}=\mathbf{k}_\textrm{in}-\mathbf{k}_\textrm{out}$. The wave vectors $\mathbf{k}$ can be described by a unit vector $\hat{\mathbf{n}}$ and wavelength

$$\mathbf{k} = \frac{2\pi}{\lambda}\hat{\mathbf{n}} \textrm{ with } ||\hat{\mathbf{n}}||_2 = 1$$

In this context we have elastic scattering, the energy does not change essentially on diffraction $||\mathbf{k}_\textrm{in}||_2 = ||\mathbf{k}_\textrm{out}||_2$, thus we can make the diffraction process completely dependent on the *scattering vector* $\mathbf{q}$.


This phenomena is sometimes also introduced equivalently using Bragg's law. You can read the equivalance proof.


<span style="color:blue"> **09** Compute the reciprocal lattice vectors.</span>

<span style="color:green"> **09** Goal: The students should remember the connection between reciprocal lattice and reflection satisfy Laue equations
</span>

In [ ]:
# set upt the code widget window
ex10_wci = WidgetCodeInput(
        function_name="reciprocal_lattice", 
        function_parameters="",
        docstring="""
Return the 2D reciprocal unit cell vectors.

:return: reciprocal lattice unit cell vectors
""",
        function_body="""

import ase
import numpy as np
from numpy import pi

# matrix-matrix and matrix-vector multiplication can be doen with @
# For example multiplying vector a with matrix R can be done with
# R @ a
R = np.array([[0,-1],[1,0]])

# 2D structure
structure = ase.Atoms(positions=[[0,0,0]], cell=[[1,0,0],[0.5,0.5,0],[0,0,0]])

# obtain 2D Bravais lattice vectors
a_1 = structure.cell[0][:2]
a_2 = structure.cell[1][:2]

### here solution
#b_1 = ...
#b_2 = ...
### BEGIN SOLUTION
b_1 = 2*pi*R @ a_2/(a_1 @ R @ a_2)
b_2 = 2*pi*R @ a_1/(a_2 @ R @ a_1)
### END SOLUTION
B = [b_1, b_2]
return B
"""
        )

data_dump.register_field("ex10-function", ex10_wci, "function_body")

In [ ]:
#for dispersion relation in diatomic lattice
def braggs_reflection(B, wavelength, n1, n2):
    if (n1==0) and (n2==0):
        return None, None, np.zeros(2)
    k_norm = 2*np.pi/wavelength
    q = n1*B[0] + n2*B[1]
    q_norm = np.linalg.norm(q)
    if k_norm < q_norm/2:
        return None, None, q
    height = np.sqrt(k_norm**2-(q_norm/2)**2)
    R = np.array([[0,-1],[1,0]])
    height_vec = R@q/np.linalg.norm(q) * height
    k_in = height_vec + q/2
    k_out = q-k_in
    return k_in, k_out, q

def plot_braggs_reflection(ax, log_wavelength, n1, n2):
    B = ex10_wci.get_function_object()()
    k_in, k_out, q = braggs_reflection(B, np.exp(log_wavelength), n1, n2)
    
    integer_lattice = list(itertools.product(list(range(10)), repeat=2))
    lattice = (np.mgrid[:30,:30].T @ B).reshape(-1, 2)
    lattice -= (np.array([15,15]).T @ B).reshape(-1, 2)

    ax.scatter(lattice[:,0], lattice[:,1], label='reciprocal lattice', s=75)


    def vector_to_arrow(vector, head_length):
        # we make the vector a bit smaller so the tip of the head of the arrow
        # points correctly to the end of the vector
        return vector * (1-head_length/np.linalg.norm(vector))


    head_length = 1
    head_width = 1

    q_arrow = vector_to_arrow(q, head_length)
    ax.arrow(0, 0, q_arrow[0], q_arrow[1], lw=3,
              head_width=head_width, head_length=head_length,
              fc='black', ec='black', label=f'{n1}$b_0$+{n2}$b_1$')

    if not(k_in is None) and not(k_out is None):
        k_in_arrow = vector_to_arrow(k_in, head_length)
        ax.arrow(0, 0, k_in_arrow[0], k_in_arrow[1], lw=3,
                  head_width=head_width, head_length=head_length,
                  fc='red', ec='red', label='k_in')
        
        k_out_arrow = vector_to_arrow(k_out, head_length)
        ax.arrow(k_in[0], k_in[1], k_out_arrow[0], k_out_arrow[1], lw=3,
                  head_width=head_width, head_length=head_length,
                  fc='orange', ec='orange', label='k_out')
        
        theta = np.linspace(-np.pi, np.pi, 200)
        r = 2*np.pi/np.exp(log_wavelength)
        ax.plot(np.sin(theta)*r + k_in[0], np.cos(theta)*r + k_in[1], color='gray', label="Ewald's sphere")

        
    else:
        print(f"The log wavelength {log_wavelength} is too large to result in a scattering vector q = {N0}b_0+{N1}b_1.")
    ax.set_xlim(-40,30)
    ax.set_ylim(-40,30)
    ax.set_xlabel("$b_1$")
    ax.set_ylabel("$b_2$")
    ax.set_aspect('equal')
    ax.legend()

In [ ]:
fig_ax = plt.subplots(1, 1, figsize=(7,6), tight_layout=True)
ex10_wp = WidgetPlot(plot_braggs_reflection, 
               WidgetParbox(log_wavelength = (0., -2.2, 0.2, 0.05, r'$\log\lambda$'),
                             n1 = (1, -5, 5, 1, r'$n_1$'),
                             n2 = (1, -5, 5, 1, r'$n_2$')), fig_ax=fig_ax)
ex10_wcc = WidgetCodeCheck(ex10_wci, ref_values = {}, demo = ex10_wp)
display(ex10_wcc)

<span style="color:blue"> **10**  For the scattering vector $\mathbf{q}=3\mathbf{b}_1+2\mathbf{b}_2$, estimate what the maximal energy of a ray can be to be reflected with this scattering vector (use the estimated wavelength with Planck's equation).</span>

# Diffraction - diffraction patterns

To understand the diffraction pattern of crystal systems we have to think of a plane of waves being reflected from a crystal, the intensity of the reflected waves depends on the inciding plane describing the wave vectors and the lattice of the crystal. Therefore we can quantify the scattering amplitude as a sum of individual contributions of scattering on each atom in the structure

$$
F(\mathbf{q}) = \frac{1}{N} \sum^N_{j \textrm{ atom}} f_j\exp(-i\mathbf{q}\mathbf{r}_j)\textrm{, with }\mathbf{r}_j\textrm{ position of atom }j
$$

with the atomic form factors $f_j$ which describes the scattering for the atom $j$. The scattering is characterized by the electronic charge density of the atom $n_j$

$$
f_j(\mathbf{q}) = \int_\mathbb{R} \mathrm{d}\mathbf{r}\, n_j(\mathbf{r}-\mathbf{r}_j)\exp(-i\mathbf{q}(\mathbf{r}-\mathbf{r}_j))
$$

which we approximate for simplicity with a point charge, so we get for the atomic form factor the number of electrons $$f_j \approx Z$$. In 2D we can express the scattering vector as $\mathbf{q} = (\sin\theta, \cos\theta)$ and plot the intensity dependent on $\theta$.

For the simulation of the diffraction experiments we will look at the 2D monoclinic crystal InAs.

In [ ]:
in_as = ase.Atoms("InAs", positions=[[0,0,0],[0.5,0.5,0]], cell=[1,1,1], pbc=[True,True,False])
in_as.positions *= 3
in_as.cell *= 3
cs = chemiscope.show([in_as],  mode="structure", settings={"structure":[{"axes":"abc","bonds":False,"unitCell":True,"supercell":{"0":5,"1":5,"2":1}}]})
display(cs)

In [ ]:
# set upt the code widget window
ex11_wci = WidgetCodeInput(
        function_name="diffraction_pattern", 
        function_parameters="q, structure",
        docstring="""
Computes the scatter intensity for the scattering vector q. 
Each atom is a scattering center.

:param q: scattering vector q = k_in-k_out
:param structure: structure on which the x-rays diffract
""",
    function_body="""
from numpy import abs as absolute
from numpy import exp

# you can define an imaginary number in python with 1j
# for example
# a = 1j # i
# a = 1+1j # 1+i
# b = absolute(a)**2 # |a|^2 = aa*

# you can access the atomic number Z with structure.numbers
# for j in range(len(structure)):
#     Z = structure.numbers[j] # atom number of atom j
#     r_j = structure.positions[j] # atom position of atom j

structure_factor = 0
# write solution here

### BEGIN SOLUTION
for j in range(len(structure)):
    # approx of atom form factors 
    f_j = structure.numbers[j]
    r_j = structure.positions[j]
    structure_factor += exp(-1j*q @ r_j)*f_j
scattered_intensity = absolute(structure_factor)
scattered_intensity /= len(structure)
### END SOLUTION

return scattered_intensity
"""
        )

data_dump.register_field("ex11-function", ex11_wci, "function_body")

In [ ]:
def add_periodic_image(structure, unit_cell_structure, direction):
    cell_length = unit_cell_structure.cell[0][0]
    periodic_image = copy.deepcopy(unit_cell_structure)
    periodic_image.positions += np.array(direction)*cell_length
    structure += periodic_image

def setup_structure(structure, M, noise_sigma):
    import itertools
    supercell = ase.Atoms()
    structure = copy.deepcopy(structure)
    for direction in itertools.product(list(range(M+1)), repeat=2):
        add_periodic_image(supercell, structure, list(direction) + [0])

    np.random.seed(0)
    supercell.positions[:, :2] += np.random.normal(size=(len(supercell), 2), scale=noise_sigma)
    return supercell

def plot_diffraction_pattern(ax, log_wavelength, M, noise_sigma):
    structure = ase.Atoms("InAs", positions=[[0,0,0],[0.5,0.5,0]], cell=[1,1,0], pbc=[True,True,False])
    in_as.positions *= 3
    in_as.cell *= 3
    
    supercell = setup_structure(structure, M, noise_sigma)
    
    theta = np.linspace(0.,np.pi,200)
    n = np.array([np.sin(theta),np.cos(theta),0])
    q = 2*np.pi*n/np.exp(log_wavelength)
    
    scattered_intensity = ex11_wci.get_function_object()(q, supercell)
    
    ax.plot(theta*180/np.pi, scattered_intensity)
    
    ax.set_xlim(0,180)
    ax.set_xlabel("theta $\\theta$ [degree]")
    ax.set_ylim(-0.5,42) 
    ax.set_ylabel("Scattering amplitude $F(\mathbf{q}(\\theta))$")
    #ax.set_aspect('equal')

In [ ]:
fig_ax = plt.subplots(1, 1, figsize=(7,6), tight_layout=True)
ex11_wp = WidgetPlot(plot_diffraction_pattern, 
               WidgetParbox(log_wavelength = (0., -2.2, 0.2, 0.05, r'$\log\lambda$'),
                             M = (8, 1, 10, 1, r'($M$, $M$, $M$) supercell'),
                             noise_sigma = (0., 0., 1., 0.1, r'$\mathcal{N}(0,\sigma)$ Gaussian $\sigma$')),
                             fig_ax=fig_ax)
ex11_wcc = WidgetCodeCheck(ex11_wci, ref_values = {}, demo = ex11_wp)
display(ex11_wcc)

<span style="color:blue"> **12** You can ignore the `Gaussian sigma` slider for now. At which ($\lambda$,$\theta$) can you observe peaks? What is the height of the peaks resemble? What do the peaks resemble?</span>

In [ ]:
ex12_txt = Textarea("", layout=Layout(width="100%"))
data_dump.register_field("ex12-answer", ex08_txt, "value")
display(ex12_txt)

<span style="color:blue"> **13** What happens when you increase the supercell?</span>

In [ ]:
ex13_txt = Textarea("", layout=Layout(width="100%"))
data_dump.register_field("ex13-answer", ex08_txt, "value")
display(ex13_txt)

<span style="color:blue"> **14** To simulate the effect of thermal fluctations we add Gaussian noise to the position of the structure. The `Gaussian sigma` slider controls how strong the Gaussian noise is (by default no noise is addded). What happens to the peaks when you increase the noise on the atom positions?</span>

In [ ]:
ex14_txt = Textarea("", layout=Layout(width="100%"))
data_dump.register_field("ex14-answer", ex08_txt, "value")
display(ex14_txt)